# HiMAP tutorial: mock community data

In [ ]:
library(himap)

Specify full main path and load raw FASTQ files. If ~ is not expanded, blastn breaks.

In [ ]:
out_path = path.expand('~/data/zheng_2015_tutorial')

In [ ]:
fastq_path = file.path(out_path, 'fastq')
fq_fwd = read_files(fastq_path, 'R1')
fq_rev = read_files(fastq_path, 'R2')

Generate output files for the pre-processing part of the pipeline:

In [ ]:
sample_ids = sampleids_from_filenames(fq_fwd, separator='_')
fq_mer = file.path(out_path, 'himap_merged', paste0(sample_ids, '.fastq'))
fq_tri = file.path(out_path, 'himap_trimmed', paste0(sample_ids, '.fastq'))
fq_fil = file.path(out_path, 'himap_filtered', paste0(sample_ids, '.fastq'))

## Pre-processing
Merge reads, trim PCR primers and perform quality control and final trimming.

In [ ]:
mergestats = merge_pairs(fq_fwd, fq_rev, fq_mer, verbose=T)

In [ ]:
mergestats

Primer set used for amplifying V3-V4 region was 341F and 805R, which are labelled as 'V3-V4-2' in our reference table:

In [ ]:
himap_option('blast_dbs')[6]

In [ ]:
trimstats = pcr_primer_trimmer(fq_mer, fq_tri, region='V3-V4-2', verbose=T)

In [ ]:
trimstats

Now find a length to trim all reads before passing them to DADA2 denoising. A good rule of thumb is length above which we have 99% of the reads:

In [ ]:
seqlen.ft = sequence_length_table(fq_tri)
trim_length = ftquantile(seqlen.ft, 0.01)
trim_length

Trim reads to this length and then filter out the remaining reads with too many (> 2) expected errors:

In [ ]:
filtstats = filter_and_trim(fq_tri, fq_fil, truncLen=trim_length)

In [ ]:
filtstats

## DADA2 denoising
Denoise these filter and trimmed reads, then pass on the pre-trimmed reads (`fq_tri`) to retrieve back full-length sequences before we align them vs a reference database:

In [ ]:
dada_result = dada_denoise(fq_fil, fq_tri, verbose=T)

Extract sequence abundance from DADA2 output:

In [ ]:
ab.dt = sequence_abundance(dada_result)

## BLAST
Align the un-trimmed sequences to the HiMAP reference database for V3-V4 region:

In [ ]:
blast_output = blast(ab.dt, region='V3-V4-2', verbose=T)

## OSU abundance table
From sequence abundance table and BLAST output, estimate the abundance of each Operational Strain Unit (OSU):

In [ ]:
osu_ab.dt = abundance(ab.dt, blast_output)

In [ ]:
osu_ab.dt

In [ ]:
## Taxonomy
Assign NCBI taxonomy classifi